## Etymology

In [3]:
import pandas as pd
import json
from fuzzywuzzy import fuzz
import re

c:\Users\eviye\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [41]:
# load data, view dataset size
et = pd.read_csv('../data/etymology.csv')
display(et.head())
print(et.shape)
eng = et[(et['lang'] == 'English')]
print(eng.shape)
eng.head()
test = eng.iloc[:eng.shape[0]//4]
test.shape

,term_id,lang,term,reltype,related_term_id,related_lang,related_term,position,group_tag,parent_tag,parent_position
0,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,group_derived_root,NaN,NaN,NaN,0,0IIU6TlkSA2D7dg1PfxGOw,NaN,NaN
1,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,borrowed_from,fhl-sdgwXaWvYD8CgaKXpQ,Latin,thēsaurus,0,NaN,0IIU6TlkSA2D7dg1PfxGOw,0.0
2,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός,0,NaN,0IIU6TlkSA2D7dg1PfxGOw,1.0
3,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,doublet_with,sqiifJX3XIOQ0vZ0trzRaA,English,treasure,0,NaN,NaN,NaN
4,iwDzanYmWxaUKbOlUy2yyQ,Latin,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός,0,NaN,NaN,NaN


(3884337, 11)
(880736, 11)


(220184, 11)

In [ ]:
# test.to_csv('Final\data\\test_english.csv')
# eng.to_csv('Final\data\\english.csv')

In [48]:
euro = {
  "Azerbaijan":'azerbaijani',
  "Albania":'albanian',
  "Armenia":'armenian',
  "Bosnia and Herzegovina":'bosnian',
  "Bulgaria":'bulgarian',
  "Cyprus":'',
  "Denmark": 'danish',
  "Ireland": 'irish',
  "Estonia": 'estonian',
  "Austria": '',
  "Czech Republic":'czech',
  "Finland":'finnish',
  "France":'french',
  "Georgia":'georgian',
  "Germany":'german',
  "Greece":'greek',
  "Croatia":'croatian',
  "Hungary":'hungarian',
  "Iceland":'icelandic',
  "Israel":'hebrew',
  "Italy":'italian',
  "Latvia":'latvian',
  "Belarus":'belarusian',
  "Lithuania":'lithuanian',
  "Slovakia":'slovak',
  "Liechtenstein":'',
  "The former Yugoslav Republic of Macedonia":'macedonian',
  "Malta":'maltese',
  "Belgium": 'flemish',
  "Faroe Islands": "faroese",
  "Andorra": '',
  "Luxembourg": 'luxembourgish',
  "Monaco": 'monegasque',
  "Montenegro":'montenegrin',
  "Netherlands": 'dutch',
  "Norway": 'norwegian',
  "Poland":'polish',
  "Portugal":'portuguese',
  "Romania":'romanian',
  "Republic of Moldova":'gagauz',
  "Slovenia":'slovene',
  "Spain":'spanish',
  "Sweden":'swedish',
  "Switzerland":'romansh',
  "Turkey":'turkish',
  "United Kingdom":'english',
  "Ukraine":'ukranian',
  "San Marino":'',
  "Serbia":'serbian',
  "Holy See (Vatican City)":'latin',
  "Russia":"russian"
}

In [49]:
euro_inverse = {v:k for k,v in euro.items() if v}
len(euro), len(euro_inverse)

(51, 46)

In [50]:
# create language link-node data
# et['lang_clean'] = et['lang'].str.strip().str.lower()
# et['related_lang_clean'] = et['related_lang'].str.strip().str.lower().fillna('')

# langs = set(et['lang_clean'])
# langs.update(et['related_lang_clean'])
# langs.remove('')
# print(langs)
langs = euro_inverse.keys()
# langs.remove('')
langs = list(langs)
nodes = [{'id': i, 'name': langs[i]} for i in range(len(langs))]
nodes[-5:]

[{'id': 41, 'name': 'english'},
 {'id': 42, 'name': 'ukranian'},
 {'id': 43, 'name': 'serbian'},
 {'id': 44, 'name': 'latin'},
 {'id': 45, 'name': 'russian'}]

In [54]:
grouped = et[(et['related_lang_clean'] != '') & (et['lang_clean'] != et['related_lang_clean'])].groupby(by=['lang_clean','related_lang_clean','reltype'])[['term']].count().reset_index()
display(grouped.head())
link_dict = {}
for i, row in grouped.iterrows():
    key = (row['lang_clean'].strip().lower(), row['related_lang_clean'].strip().lower())
    if (key[0] not in euro_inverse) or (key[1] not in euro_inverse):
        continue
    if key[::-1] in link_dict: # reverse direction is already in the dict
        key = key[::-1]
    if key not in link_dict: # if pair is new to the dictionary
        link_dict[key] = {'source': langs.index(key[0]), 'target': langs.index(key[1])}
    if row['reltype'] not in link_dict[key]:
        link_dict[key][row['reltype']] = 0
    link_dict[key][row['reltype']] += row['term'] # add the current term count to the dictionary

,lang_clean,related_lang_clean,reltype,term
0,'are'are,are'are,compound_of,6
1,'are'are,hawaiian,cognate_of,2
2,'are'are,maori,cognate_of,2
3,'are'are,proto-austronesian,etymologically_related_to,2
4,'are'are,proto-austronesian,inherited_from,7


In [55]:
links = list(link_dict.values())
for link in links:
    total = sum([val for key, val in link.items() if (key not in ['source', 'target'])])
    link['value'] = total
links[:5]

[{'source': 1,
  'target': 2,
  'cognate_of': 42,
  'derived_from': 1,
  'etymologically_related_to': 1,
  'value': 44},
 {'source': 1, 'target': 0, 'cognate_of': 1, 'value': 1},
 {'source': 1,
  'target': 20,
  'cognate_of': 2,
  'etymologically_related_to': 1,
  'value': 3},
 {'source': 1,
  'target': 4,
  'borrowed_from': 17,
  'cognate_of': 86,
  'derived_from': 8,
  'etymologically_related_to': 13,
  'value': 124},
 {'source': 1, 'target': 8, 'cognate_of': 11, 'derived_from': 1, 'value': 12}]

In [57]:
data_out = {'country_to_lang':euro_inverse,
            'nodes':nodes,
            'links':links}
with open('../data/lang_links.json', "w", encoding="utf-8") as outfile: # even if there's an error, 
        outfile.write(json.dumps(data_out, indent=4))

In [8]:
# language to country map - IGNORE EVERYTHING HERE AND BELOW
with open('../data/lang_to_country.json', 'r') as f:
    langs = json.load(f)
langs = [lang['lang'] for lang in langs]
# lang_country = {}
# for lang in langs:
#     country = input(lang+': ').strip()
#     lang_country[lang] = country

In [35]:
countries = pd.read_csv("../data/List_of_official_languages_by_country_and_territory_1.csv").set_index("Country/Region")
countries.head()
countries_dict = {}
for country, row in countries.iterrows():
    if country in euro:
        country_langs = []
        for lang in row[~row.isna()].values:
            country_langs.extend([re.sub(r'\s*\(.*\)\s*', '', lang.lower()) for lang in lang.split('\n')])
        countries_dict[country] = country_langs
# countries_dict = {country: list(row[~row.isna()].values) }
countries_dict


{'Albania': ['albanian', 'greek', 'macedonian', 'aromanian', 'italian'],
 'Andorra': ['catalan', 'spanish', 'french', 'portuguese'],
 'Armenia': ['armenian', 'armenian'],
 'Austria': ['german',
  'burgenland croatian',
  'hungarian',
  'slovene',
  'slovene',
  'czech',
  'hungarian',
  'slovak',
  'romani',
  'serbian',
  'german'],
 'Azerbaijan': ['azerbaijani', 'azerbaijani'],
 'Belarus': ['belarusian', 'russian', 'belarusian'],
 'Belgium': ['dutch', 'french', 'german', 'dutch', 'french', 'german'],
 'Bosnia and Herzegovina': ['none'],
 'Bulgaria': ['bulgarian'],
 'Croatia': ['croatian',
  'serbian',
  'hungarian',
  'czech',
  'slovak',
  'pannonian rusyn',
  'istro-romanian',
  'italian',
  'romani',
  'slovene'],
 'Cyprus': ['greek', 'turkish', 'armenian', 'cypriot arabic', 'english'],
 'Czech Republic': ['czech',
  'slovak',
  'belarusian',
  'bulgarian',
  'croatian',
  'german',
  'greek',
  'hungarian',
  'polish',
  'romani',
  'russian',
  'rusyn',
  'serbian',
  'ukrainian

In [23]:
def evaluate_langs (options, target):
    target = target.replace('old ', '').replace('middle ', '').replace('high ', '').replace('middle ', '').replace('low ', '')
    score = 0
    for i in range(len(options)):
        score += 1/((i+1))*fuzz.partial_ratio(options[i], target)
    return score
lang_country = {}
for lang in langs:
    lang_country[lang] = max(countries_dict, key=lambda country: evaluate_langs(countries_dict[country], lang))


In [24]:
lang_country

{'latin': 'Italy',
 'middle english': 'Sweden',
 'proto-indo-european': 'Romania',
 'old english': 'Sweden',
 'ancient greek': 'Cyprus',
 'old french': 'France',
 'french': 'France',
 'proto-germanic': 'Germany',
 'german': 'Germany',
 'dutch': 'Netherlands',
 'middle french': 'France',
 'swedish': 'Sweden',
 'italian': 'Italy',
 'old norse': 'Norway',
 'danish': 'Sweden',
 'icelandic': 'Germany',
 'scots': 'Croatia',
 'spanish': 'Germany',
 'proto-west germanic': 'Germany',
 'late latin': 'Italy',
 'west frisian': 'Germany',
 'medieval latin': 'Italy',
 'anglo-norman': 'Romania',
 'arabic': 'Romania',
 'middle dutch': 'Netherlands',
 'low german': 'Germany',
 'old high german': 'Germany',
 'sanskrit': 'Germany',
 'saterland frisian': 'Germany',
 'norwegian': 'Norway',
 'japanese': 'Spain',
 'middle low german': 'Germany',
 'russian': 'Belarus',
 'german low german': 'Austria',
 'hebrew': 'Israel',
 'middle high german': 'Germany',
 'portuguese': 'Portugal',
 'frankish': 'Germany',
 'i

In [12]:
evaluate_langs(countries_dict['Czech Republic'], 'latin')

4442

## Wordnet - not using anymore

Wordnet documentation: https://www.nltk.org/api/nltk.corpus.reader.wordnet.html#module-nltk.corpus.reader.wordnet

In [2]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eviye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\eviye\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
counts = set()
wordnet.lemmas('run')[0].attributes()
for lemma in wordnet.lemmas('run'):
    counts.add(lemma.count())
counts

print(wordnet.lemmas('run')[0].synset().definition())
print(wordnet.lemmas('run')[0].derivationally_related_forms()[0].synset().definition())

a score in baseball made by a runner touching all four bases safely
make without a miss
